# Executable Categories, Executing Categories

What does is take for something to be considered "real"? People are different, and that's nice. Different people have different requirements. 

A formal logic consisting of manipulation of syntax makes people quasy until they have understanding of the amthemtical semantics .
A mathematical theory makes some people quesy until it can be applied in the sciences and engineering.
Programming languages make people quesy until they can see how the features are implementable. Others are quesy until they can see a purely mathemtical model of the features

Figuring out these bridges can be a nontrivial undertaking.
Once these things are seen, going the other direction evokes a sensation of " why are you making things so complicated, so abstract?"

The criteria I have for whether something makes sense tends to be "Can I implement something at least evocative of this theory that I can execute as a computer program"

I personally am not particularly satsified by merely reprsenting the syntactical structure on the computer.


There is a pump of undertsanding of concretization and abstraction.

## Bicategory of Relations/Allegories
Effective set representation >= effective relation representation
There is finetuning (Set (a,b) vs Map a (Set b))  , but in broad strokes I'm not sure I know of any relation reprsentations that don't have set-like analogs.

Abstract Interpretation. Domains and Lattices.
Algerba of Programming book
Closed form solutions vs open form.


People think of blobs (~ Geomeotry) and finite sets when they think of sets casually. But there is great importance and power in noting what is 


### Linear
  

#### Polynomial
   Circuits and Feedback
### Polyhedral

### Algebraic

## Categorifying symbolic DSLS

### Cvxpy & JuMP
   petri nets. Convex relations
### Graphviz
### Z3py
### Sympy

## Solvers for Abstract Categories

### Autmated Theorem Provers
### Logic Programming

### Knuth Bendix
### SAT for finite categories


## WIP

### Why Julia

- Julia is acceptable to the scientfic and engieering community. Haskell is not. I was entranced by Haskell, but I am an irresponsible POS
- Catlab has critical mass
- Julia has the ability tyo implement novel algorithms at acceptable speed.
- Community with really interesting ideas and open to others.

### Why Categories
- A uniform system that spans 
- 

### Gaussians

### AD lens
Are lenses special.

There is one persepctive that lenses lie on the spectrum of coroutines in which the control flow is encoded statically in the type.

Triple pass lenses
Infinite lenses

Imperative staetful programming is extremely powerful
Gotos are extremely powerful.
Buit these things live in chaos
The point is to cut a sublte line

continuations are an overly powerful abstraction. They are functional gotos.


Wengert Tapes
Closures 
  
  
  Uniformity
  Interval Arithmetic
  
  
  
### Algebra of Programming Solver

A tasty little sampler, like you're right there in the Applebee's



I've stripped annotations and features to give a minimal presentation, not necessarily a best one. 
I am bending Julia best practices and idioms.

If you're interested in running this notebook online in Google Colab, run this to gt Julia running

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.5.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CuArrays"
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

# Computer Functions as a category

- The usual one. Morphisms are functions, objects are types


Base strcutural combinators +
 a piles of domain specific atoms

In [29]:
module Func1

id = x -> x
compose(f,g) = x -> g(f(x))
otimes(f,g) = ((x,y),) -> (f(x),g(y))
mcopy = x -> (x,x)
fst = ((x,y),) -> x
snd = ((x,y),) -> y
pair(f,g) = x -> (f(x), g(x))
assoc = (((x,y),z),) -> (x, (y,z))
braid = ((x,y),) -> (y,x)

add = ((x,y),) -> x + y # xy -> +(xy...)
mul = ((x,y),) -> x * y # xy -> *(xy...)
#spread(f) = (x,) -> f(x...)
square = compose(mcopy, mul)

[
    compose(fst,snd)(((:a, :c), :b)),
    compose(fst, id)((:a, :b)),
    id(:a),
    otimes(id,id)((:a, :b))
]
end

LoadError: invalid redefinition of constant id

In [27]:
[
    compose(fst,snd)(((:a, :c), :b)),
    compose(fst, id)((:a, :b)),
    id(:a),
    otimes(id,id)((:a, :b))
]

4-element Array{Any,1}:
 :c
 :a
 :a
 (:a, :b)

### Flat representation

- There are always choices in representation
- On the nose associativity via storing in arrays
- caonical form is nice, less assoc jiggling
- Flatten pointer indirections
- We run into problems where the data we need just isn't there

Using tuples, we represented the monoidal product, however it can be useful to flatten this structure, making the product associative on the nose rather thanm just an isomorphism. This gives useful canonical forms and flattens an unnecessary pointer indirection.

It is not all rosy though.

In [ ]:
id = x -> x
compose(f,g) = x -> g(f(x))
fst = () # ???
otimes(f,g) = () #  ???


Missing information:

choice 1. user annotate everything with types.   
pair(a,b,c,f,g)

choice 2. wrapper types.
struct FunMorph
   dom
   cod
   fun
end

choice 3 type level shenanigans
4. others

In [ ]:
struct Func
   fun
   dom 
   cod
end

id(m) = Func(x -> x, m, m)
compose(f,g) = Func(x -> g(f(x)), f.dom, g.cod)

fst(m,n) = Func(x -> x[1:m], m + n, m)
snd(m,n) = Func(x -> x[end-n:end], m + n , n

otimes(f,g) = Func(  )

### Comment - 
  multiple dispatch
  struct FuncOb
    m::Int64
  end
  
  Catlab


In [11]:
# junk
id = (x...) -> x
compose(f,g) = (x...) -> begin
       y = f(x...)
       y isa Tuple ? g(y...) : g(y)
end
      # g(f(x...)...)
otimes(f,g) = (x,y) -> (f(x),g(y))
mcopy = x -> (x,x)
fst = (x,y) -> x
snd = (x,y) -> y
pair(f,g) = x -> (f(x), g(x))
assoc = ((x,y),z) -> (x, (y,z))

compose(fst,snd)(((:a,), (:c,)), (:b,))

compose(fst, id)((:a,), (:b,))
#id(:a)

#47 (generic function with 1 method)

# Matrices as a category. FinVect

- Linear Operators are the morphism represented by matrices
- Objects are vector spaces represented by an integer of dimensions.

- 2 common choices for a monoidal product
  - Direct product - This why lies quantum mechanics and probability
  - Direct Sum - Considered more rarely. More approrpate for linear dynamical systems.
  
  
There is a well known category of matrices that uses the Kronecker product as it's monoidal product. 
I am going to use an alternative monoidal product though, the direct sum.

https://github.com/AlgebraicJulia/Catlab.jl/blob/master/src/categorical_algebra/Matrices.jl
# Block diagonal only implemented for sparse matrices.
blockdiag(A::AbstractMatrix...) = cat(A..., dims=(1,2))

In [ ]:
using LinearAlgebra
eye(n) = Matrix{Float64}(I,n,n)

In [20]:


id(n) = eye(n) 
compose(f,g) = g * f
function otimes(f,g) 
    (n,m) = size(f)
    (p,q) = size(g)
    [     f      zeros(n,q) ;
      zeros(p,m)     g      ]
end
mcopy(n) = [ eye(n) ;
             eye(n) ] 
fst(n,m) = [   eye(n)   zeros(n, m)] 
snd(n,m) = [zeros(m, n)    eye(m)  ] 
             
pair(f,g) = [ f ;
              g ]

sum(n) = tr(mcopy(n))

sum (generic function with 1 method)

In [17]:
fst(2,3)

2×5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0

In [13]:
examples = :([
    compose(id(2),id(2)),
    #compose(fst, id),
    id(2),
    otimes(id(2),id(3))
        ] )
for example in examples.args
    println("$example = $(eval(example))")
    display(eval(example))
end

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

5×5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0

compose(id(2), id(2)) = [1.0 0.0; 0.0 1.0]
id(2) = [1.0 0.0; 0.0 1.0]
otimes(id(2), id(3)) = [1.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 1.0]


## A functional representation for linear maps as a category
- Linear functions are a subcategory of the category of functions, i.e. only those who's objects are vector spaces and whose maps are linear.
- The same combinators as before work
While the most elementary way to implement a linear operator is the have a dense 2 dimensional array that is the matrix of that operation, there are other options. Perhaps our operator is sparse, in which case we may want to store a sparse matrix. Another possibility is that our operation is simple to specify in some algorithmic sense, for example the Discrete Fourier Transform can be described as a matrix, but really it is most efficient to describe it as a Fast Fourier Transform subroutine to be called. So we can describe This functional representation is not weird, it is in scipy, which is as mainstream as it gets in scientific computing 

https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.LinearOperator.html#scipy.sparse.linalg.LinearOperator
https://github.com/JuliaSmoothOptimizers/LinearOperators.jl



In [ ]:
struct Func
   fun
   dom 
   cod
end

id(m) = Func(x -> x, m, m)
compose(f,g) = Func(x -> g(f(x)), f.dom, g.cod)

fst(m,n) = Func(x -> x[1:m], m + n, m)
snd(m,n) = Func(x -> x[end-n:end], m + n , n

otimes(f,g) = Func(  )

# Automatic differentiation as a category
### Bundle together nonlinear functions with their Jacobian matrices


### Association, Matrix product problems and "Hughes Lists"




## Comments
# Dual numbers

# Cotangent spaces
### How exotic is this
It's not. The point of putting the backwards pass function where it is to to close over the input variable. If you closure convert the lens, it takes this form
x -> (e,y)
(e,dy) -> dx
This is the exitensital form of a lens
Exsitetnail types have something to do with object oriented programming. backwards : e -> dy -> dx

If you defunctionalize instead you get a data structure. This is a Wengert tape data structure.

### Lenses and continuations
From this perspective, Lenses live on a spectrum of control flow techniques.


(x -> (y, dy -> dx))
~  (x -> (y, forall s. (dx -> s) -> (dy -> s)) 
~> forall s. (x , dx -> s) -> (y , dy -> s)

What this is evocative of is that reverse mode AD is a map pulling back linear functionals (dx -> R). Analog of dual number formulation (x,dx) of forward mode AD.

https://twitter.com/SandMouth/status/1270409619693875201?s=20

## References
http://conal.net/papers/essence-of-ad/
https://www.philipzucker.com/reverse-mode-differentiation-is-kind-of-like-a-lens-ii/
https://t.co/4tjLhB4b4P?amp=1
Keno 

https://www.youtube.com/watch?v=17gfCTnw6uE&feature=youtu.be&ab_channel=Topos Good lord. They really covered the gamut

In [ ]:
id = x -> (x, dx -> dx)
fst(a,b) = x -> (x[1:A.m] , dx -> [dx ; zeros(B.m)  ])
snd(a,b) = x -> (x[end-B.m:end] , dx -> [zeros(B.m) ; dx] )
function pair(f::Lens,g::Lens)
    @assert f.dom == g.dom
    x -> begin
    (y, df) = f(x)
    (z, dg) = g(x)
    (vcat(y,z), dq -> df(dq[1:f.cod]) + dg(dq[end-g.cod:end])   )

                        ( x[1:f.dom]  )
    end
end
compose(f::Lens, g::Lens) = 
    x -> begin
         (y, df) = f.it(x)
         (z, fg) = g.it(y)
         (z, df ∘ dg)
         end

mcopy(A::LensDomain) = x -> (vcat(x,x), dxx -> dxx[1:A.m] + dxx[A.m+1:end] )
otimes(f,g) = xy -> begin
                (c, df) = f(xy[1:f.dom.m])
                (d, dg) = g(xy[end-g.dom.m:end  ])
                dfg = dcd -> begin 
                             dx = df(dxd[]) 
                             dy = dg(dcd[])
                             vcat(dx,dy)
                             end
                ([c ; d] , dfg )
end

# the lens domain isn't really necessary. I guess it might changed the stage? Dimension is now known before x values rather than at the same time. That's nice
mul(n) = x -> (x[1:A.m] .* x[A.m+1:end] , dx -> vcat( dx .* x[A.m+1:end], dx .* x[1:A.m]  ) )
add(n) = x -> (x[1:A.m] .+ x[A.m+1:end] , dx -> vcat( dx , dx ) ) # sum and dup are dual.


#sum(  )
#fold



sin(A::LensDomain) = x -> (sin.(x) , dx -> cos.(x) .* dx)
cos(A::LensDomain) = x -> (cos.(x) , dx -> -sin.(x) .* dx)
pow(A::LensDomain, n) = x -> (x ^ n, dx -> n * dx .* x ^ (n-1) )

# Finite Relations
- Objects = finite sets
- Morphisms = Relationships between these sets ~ Sets of tuples
- Simplest representation: Array of tuples

In [23]:
id(m) = [(x,x) for x in m ]
compose(f,g) = [(a,c) for (a,b1) in f for (b2,c) in g if b1 == b2 ]
fst(a,b) = [   for x in a for y in b ] 


converse(f) = [(y,x) for (x,y) in f ]
x ⊆ y = all( [(a,b) ∈ y for (a,b) in x] )
meet
join


LoadError: syntax: unexpected "]"

In [22]:
methods(⊆)

# 4 methods for generic function "issubset":
[1] issubset(a::BitSet, b::BitSet) in Base at bitset.jl:419
[2] issubset(r::Base.OneTo, s::Base.OneTo) in Base at range.jl:897
[3] issubset(r::AbstractUnitRange{var"#s91"} where var"#s91"<:Integer, s::AbstractUnitRange{var"#s90"} where var"#s90"<:Integer) in Base at range.jl:899
[4] issubset(l, r) in Base at abstractset.jl:265

## Other representations
Sets of tuples
BDDS
iterator functions a -> [b]

DataFrames

composition is an inner join

This is a sprase array representation of a boolean valued matrix.
You can consider a dense matrix, and it is conceptually convenient to do so. Perhaps computationally convenenit
interval pacing and adjunctions


https://www.cambridge.org/core/books/relational-mathematics/8CB9CE4F196319222E8991D909AA2F87 - Relational Mathematics - Gunther Schmidt
Pixel Arrays - http://math.mit.edu/~dspivak/informatics/PixelArrays--SpivakDobsonKumari.pdf 
https://www.philipzucker.com/a-short-skinny-on-relations-towards-the-algebra-of-programming/
http://www4.di.uminho.pt/~jno/ps/pdbc.pdf
https://themattchan.com/docs/algprog.pdf
http://www.philipzucker.com/computational-category-theory-in-python-i-dictionaries-for-finset/ - A similar approach for FinSet
relmics


# Linear Relations

subclasses of all representations
Linear subspace dually described by generators and relations. Nullspace vs range characterization

Lattices
- Intervals
- Octagons
- Polyhedra
- Convex Sets
- Linear Subspaces


Why?
Control Systems
Open Linear Equations
Circuits - Network
A pile of linear equations more directly maps into this than the linear operator concept.


In [ ]:
id(n) = [eye(n) -eye(n)]

struct LinRel
    left
    right
end


id(n) = LinRel(eye(n), -eye(n))


function compose(x::LinRel,y::LinRel) 
    # extract sizes of different matrices
    (m, n) = size(x.left)
    (n1, x) = size(x.output)
    @assert n1 == n
    ny, yi = size(y.input)
    ny1, yo = size(y.output)
    @assert ny1 == ny
    
    # combine constraints
    A = [ x.input      x.output zeros(nx,yo) ;
          zeros(ny,xi) y.input  y.output     ]
    # convert to range representation
    B = nullspace(A)
    # project in range representation
    projB = [B[1:xi       ,:] ;
             B[xi+yi+1:end,:] ]
    # return to nullspace representation
    C = Base.transpose(nullspace(Base.transpose(projB)))
    return LinRel( C[:, 1:xi] ,C[:,xi+1:end] )
end

# basically the direct sum. The monoidal product of linear relations
function otimes( x::LinRel{T}, y::LinRel{T}) where {T} 
    nx, xi = size(x.input)
    nx1, xo = size(x.output)
    @assert nx1 == nx
    ny, yi = size(y.input)
    ny1, yo = size(y.output)
    @assert ny1 == ny
    return LinRel(    [ x.input                zeros(nx,yi);
                       zeros(ny,xi)           y.input               ],
                      [x.output               zeros(nx,yo);
                       zeros(ny,xo)     y.output               ])
    
end


In [ ]:
# generic linear relations

# Objects are the dimensionality of the vector space
struct LinRelOb{T}
    m::Int
end

# Linear relations represented 
struct LinRel{T}
  input::AbstractArray{T,2}
  output::AbstractArray{T,2}
end



dom(x::LinRel{T}) where T = LinRelOb{T}(size(x.input)[2])
codom(x::LinRel{T}) where T = LinRelOb{T}(size(x.output)[2])
id(X::LinRelOb{T}) where T = LinRel{T}(Matrix{T}(I, X.m, X.m), -Matrix{T}(I, X.m, X.m))


plus(X::LinRelOb{T}) where {T} = LinRel{T}( [Matrix{T}(I, X.m, X.m) Matrix{T}(I, X.m, X.m)] , - Matrix{T}(I, X.m, X.m) )


mcopy(X::LinRelOb{T}) where {T} = LinRel{T}( [Matrix{T}(I, X.m, X.m) ; Matrix{T}(I, X.m, X.m)] , -Matrix{T}(I, 2*X.m, 2*X.m) ) # copy input
delete(A::LinRelOb) = LinRel( zeros(0,A.m) , zeros(0,0) )   
create(A::LinRelOb) = LinRel( zeros(0,0) , zeros(0,A.m) )
dagger(x::LinRel) = LinRel(x.output, x.input)

dunit(A::LinRelOb) = compose(create(A), mcopy(A))
dcounit(A::LinRelOb) = compose(mmerge(A), delete(A))


#scale(M) = DynMorph( mat1(M),mat1(-1))
#diff =  scale(i*s) #DynMorph(mat1(i*s) , mat1(-1)) # could also lift to m dim like idd # scale(i*s)
#integ = dagger(diff)


function compose(x::LinRel,y::LinRel) 
    nx, xi = size(x.input)
    nx1, xo = size(x.output)
    @assert nx1 == nx
    ny, yi = size(y.input)
    ny1, yo = size(y.output)
    @assert ny1 == ny
    A = [ x.input      x.output zeros(nx,yo) ;
          zeros(ny,xi) y.input  y.output     ]
    B = nullspace(A)
    projB = [B[1:xi       ,:] ;
             B[xi+yi+1:end,:] ]
    C = Base.transpose(nullspace(Base.transpose(projB)))
    return LinRel( C[:, 1:xi] ,C[:,xi+1:end] )
end

# basically the direct sum. The monoidal product of linear relations
function otimes( x::LinRel{T}, y::LinRel{T}) where {T} 
    nx, xi = size(x.input)
    nx1, xo = size(x.output)
    @assert nx1 == nx
    ny, yi = size(y.input)
    ny1, yo = size(y.output)
    @assert ny1 == ny
    return LinRel(    [ x.input                zeros(nx,yi);
                       zeros(ny,xi)           y.input               ],
                      [x.output               zeros(nx,yo);
                       zeros(ny,xo)     y.output               ])
    
end

Lay out all at once vs incremnetal - quantifier elimination

## Point Freeing Pointful DSLs

# Optimization Problems

optimization problems are interesting. This is one formulation that is very natural.
Convex operators.
Fermat's principle, principle of least action

min_y f(x,y) + g(y,z)

Steepest descent
idea : Relations (feasible set) + additive objective functions

http://www.philipzucker.com/a-sketch-of-categorical-relation-algebra-combinators-in-z3py/
http://www.philipzucker.com/categorical-combinators-for-convex-optimization-and-model-predictive-control-using-cvxpy/

# Graphs
Maybe. Basically the graphviz thing

http://www.philipzucker.com/categorical-combinators-for-graphviz-in-python/

# Bits and Bobbles

Polynomial linear relations
Theorem proving
Polyhedral relations
Lens for real numbers
iLQR
probabilistic programming
Quadratic Optimization and gaussian integrals + delta functions
